In [15]:
import yfinance as yf
import pandas as pd
import polars as pl
import os
import re
import urllib.parse
from datetime import datetime
import matplotlib.pyplot as plt
import statsmodels.api as sm
import math 
import pyarrow
import numpy as np
from io import StringIO
import time
import logging

Collect scores from hard memory


In [2]:
root = r'C:\Users\310\Desktop\Progects_Py\Parsim-sec\src\Analysis\data\snp_500_scores'

ticker_symbol_pattern = re.compile(r'^([A-Z]+(?:\.[A-Z]+)?)\.parquet$')

df_dict_scores = {}

for dir in os.listdir(root):
    
    match = ticker_symbol_pattern.search(dir)

    ticker = match.group(1)
    ticker = ticker.replace('.', '-')
    print(ticker)

    file_path = os.path.join(root, dir)

    df = pl.read_parquet(file_path)
    df_dict_scores[ticker] = df 


A
AAL
AAPL
ABBV
ABNB
ABT
ACGL
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIZ
AJG
AKAM
ALB
ALGN
ALL
ALLE
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMZN
ANET
ANSS
AON
AOS
APA
APD
APH
APTV
ARE
ATO
AVB
AVGO
AVY
AWK
AXON
AXP
AZO
BA
BAC
BALL
BAX
BBWI
BBY
BDX
BEN
BF-B
BIIB
BIO
BK
BKNG
BKR
BLDR
BLK
BMY
BR
BRK-B
BRO
BSX
BWA
BX
BXP
C
CAG
CAH
CARR
CAT
CB
CBOE
CBRE
CCI
CCL
CDNS
CDW
CE
CEG
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNC
CNP
COF
COO
COP
COR
COST
CPAY
CPB
CPRT
CPT
CRL
CRM
CSCO
CSGP
CSX
CTAS
CTLT
CTRA
CTSH
CTVA
CVS
CVX
CZR
D
DAL
DAY
DD
DE
DECK
DFS
DG
DGX
DHI
DHR
DIS
DLR
DLTR
DOC
DOV
DOW
DPZ
DRI
DTE
DUK
DVA
DVN
DXCM
EA
EBAY
ECL
ED
EFX
EG
EIX
EL
ELV
EMN
EMR
ENPH
EOG
EPAM
EQIX
EQR
EQT
ES
ESS
ETN
ETR
ETSY
EVRG
EW
EXC
EXPD
EXPE
EXR
F
FANG
FAST
FCX
FDS
FDX
FE
FFIV
FI
FICO
FIS
FITB
FMC
FOXA
FRT
FSLR
FTNT
FTV
GD
GE
GEHC
GEN
GILD
GIS
GL
GLW
GM
GNRC
GOOGL
GPC
GPN
GRMN
GS
GWW
HAL
HAS
HBAN
HCA
HD
HES
HIG
HII
HLT
HOLX
HON
HPE
HPQ
HRL
HSIC
HST
HSY
HUBB
HUM
HWM
IBM
ICE
IDXX
IEX
IFF
ILMN
IN

This is support func to find end prices and their dates. It takes as an arguments starting index, which is the nuber of row of hist dataframe and this hist dataframe itself. Hist dataframe contain daily OHLC data for particular ticker (company). 

It also cheks wether index of end price is in hist, meaning that if this particular report was released very resently (less then 8 trading days ago) it will append none values for end prices out of range of hist df instead of raising an error.

It returns lists of end prices and corresponding dates (none values for prices and their dates, if they are out of range of hist df).

In [3]:
def find_end_price(start_index, hist):
    end_price_list = []
    end_price_date_list = []
    
    for x in range(2, 8):
        idx = start_index + x
        if idx < len(hist):
            end_price_list.append(hist.iloc[idx]['Open'])
            end_price_date_list.append(hist.index[idx])
        else:
            end_price_list.append(None)
            end_price_date_list.append(None)
    return end_price_list, end_price_date_list

This is support func to calc the regular returns. It takes start price and end_price_list as an arguments. It checks whether value in end price list is none and if it is, it appends none in returns list, instead of raising an error. So we are shure that all end price lists are of the same length, but for those prices that are not found in historical data we have none values for returns.

In [4]:
def regular_returns(start_price, end_price_list):
    retuns = []
    for end_price in end_price_list:
        if end_price is None:
            retuns.append(None)
        else:
            ret = (end_price - start_price) / start_price * 100
            retuns.append(ret)
    return retuns

This is the support func that calc SnP500 returns. It takes_start date and end_price_date_list as and arguments and then calculate returns for this timeframes. This func also checks whether value of end_price_date_list is none (meaning that it is not found in historical data), and if it is, func will append none instead of raising an error.

Now we are shure that list of snp_returns will be of equal length for all report release dates and all time frames, but for those returns that cannot be calculates due to non existent end prices we will have nones 

In [5]:
def snp_500_return(start_date, end_price_date_list, snp_price):

    snp_returns = []

    for end_date in end_price_date_list:
        if end_date is None:
            snp_returns.append(None)

        else: 
            start_index = snp_price.index.get_loc(start_date)
            start_price = snp_price.iloc[start_index]['Open']
    
            end_index = snp_price.index.get_loc(end_date)
            end_price = snp_price.iloc[end_index]['Open']
    
            ret = (end_price - start_price) / start_price * 100
            
            snp_returns.append(ret)

    return snp_returns

This is suppurt function that retrive the end quarter price. It take as an argument hist - historical price dataframe, date_str - the date of release of this particular report, df - company dataframe, with cols named with dates of releases of corresponding reports.

It takes the release date for current report and checks whether it is the last date in time series or not. If it is (this mean that we are are realy close to the end of hist df) we consider the end of hist df as the end of quarter. if it is not, we take next date in company df (which is the date of releas of the next quarterly report) and see its open price, this is pricisely the end of the quarter.

In [6]:
def find_end_quarter(hist, date_str, start_date, df):

    col_names = df.columns
    current_date_index = col_names.index(date_str)

    if current_date_index < len(col_names) - 1:

        next_date = col_names[current_date_index + 1]

        next_date_time_stemp = pd.Timestamp(next_date, tz='America/New_York')

        while next_date_time_stemp not in hist.index:

            next_date_time_stemp += pd.Timedelta(days=1)
    
        end_quarter_index = hist.index.get_loc(next_date_time_stemp)
        end_quarter_price = hist.iloc[end_quarter_index]['Open']
        end_quarter_date = hist.index[end_quarter_index]

        start_quarter_index = hist.index.get_loc(start_date)

        quarter_length = end_quarter_index - start_quarter_index

    else:
        end_quarter_price = None
        end_quarter_date = None
        quarter_length = None
    

    return end_quarter_price, end_quarter_date, quarter_length

This is a func that takes as an input a key-value pair from df_dict_scores and calculate 2-days, 3-days, 4-days, 5-days, 6-days, 7-days and full quarter excess returns for it (S&P500 is the benchmark). It stores the result as a polars df with dates of starting of the period as col names and corresponding returns as col values (seven in each: starting from 2-days down to 7-days and full quarter) - 

In [7]:
def computing_returns(tic_sym, df):

    company_ticker = yf.Ticker(tic_sym)
    hist = company_ticker.history(period="max")

    snp500_ticker = yf.Ticker("^GSPC")
    snp_price = snp500_ticker.history(period="max", auto_adjust=True)
    
    returns = {}

    for date_str in df.columns:

        start_date = pd.Timestamp(date_str, tz='America/New_York')
        
        while start_date not in hist.index:

            start_date += pd.Timedelta(days=1)
        

        start_index = hist.index.get_loc(start_date)

        start_price = hist.iloc[start_index]['Open']

        end_price_list, end_price_date_list = find_end_price(start_index, hist)
        
        if None not in end_price_list:
            end_quarter_price, end_quarter_date, length_of_quarter = find_end_quarter(hist, date_str, start_date, df)
            
        else:
            end_quarter_price = None
            end_quarter_date = None

        
        end_price_list.append(end_quarter_price)
        end_price_date_list.append(end_quarter_date)
        
        reg_returns = regular_returns(start_price, end_price_list)
        snp_returns = snp_500_return(start_date, end_price_date_list, snp_price)
        
        excess_returns = [a - b if a is not None and b is not None else None for a, b in zip(reg_returns, snp_returns)]
        
        #normalization: divide returns for each time frame by number of trading days (2-days returns/2, 3-days returns/3, ful quarter returns/length_of_quarter)
        timeframe_length = [2, 3, 4, 5, 6, 7, length_of_quarter]

        normalized_excess_returns = [x / y if x is not None and y is not None else None for x, y in zip(excess_returns, timeframe_length)]

        returns[date_str] = normalized_excess_returns

    returns = pl.DataFrame(returns)
    
    return returns


In [123]:
#example ticker symbol and correponding df
tic_sym = "ETR" #A CCI ETR DXCM
df = df_dict_scores[tic_sym]
df

2019-02-26,2019-05-03,2019-08-06,2019-11-05,2020-02-21,2020-05-11,2020-08-05,2020-11-04,2021-02-26,2021-05-06,2021-08-06,2021-11-05,2022-02-25,2022-05-05,2022-08-04,2022-11-03,2023-02-24,2023-05-04,2023-08-03,2023-11-02
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-0.255306,-0.16469,-0.1504,-0.226037,-0.307045,-0.185714,-0.255521,-0.253968,-0.33024,-0.241379,-0.203791,-0.223543,-0.342922,-0.284091,-0.243003,-0.229296,-0.331853,-0.153611,-0.151989,-0.218458
-1263.0,-125.0,-188.0,-316.0,-1495.0,-182.0,-405.0,-512.0,-1993.0,-259.0,-301.0,-376.0,-2122.0,-300.0,-382.0,-407.0,-2242.0,-151.0,-214.0,-374.0
0.452186,0.514083,0.47309,0.472521,0.460946,0.490548,0.438076,0.410193,0.454856,0.473056,0.43738,0.429535,0.449121,0.509041,0.479608,0.475026,0.455583,0.511905,0.451878,0.449794
11254.0,2245.0,3015.0,3336.0,11065.0,2569.0,3424.0,3807.0,13330.0,2774.0,3419.0,3819.0,13457.0,3125.0,4551.0,5069.0,15334.0,3182.0,3850.0,4372.0
168747.0,29754.0,43784.0,49242.0,160811.0,34879.0,53547.0,62913.0,195291.0,41950.0,56077.0,64775.0,199557.0,42001.0,65999.0,74260.0,223235.0,41924.0,58398.0,69471.0


In [95]:
computing_returns(tic_sym, df)

2019-02-25,2019-05-03,2019-07-31,2019-11-04,2020-03-10,2020-05-01,2020-07-31,2020-10-28,2021-02-22,2021-05-03,2021-08-06,2021-11-05,2022-02-22,2022-04-29,2022-08-05,2022-11-04,2023-02-24,2023-05-03,2023-08-02,2023-11-01
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-0.511661,0.356548,1.491884,-1.802115,0.22012,0.197961,-2.004385,0.14459,-1.062709,-1.220925,0.335601,-0.631741,0.628197,-1.328406,0.474296,-0.814031,-0.825341,0.106179,-1.083098,1.649153
-0.273397,0.277791,1.744018,-1.092501,2.276396,0.306058,-1.230241,0.240716,-1.065516,-0.849701,-0.333304,-0.00309,0.960589,-0.703122,0.495137,-0.42485,-0.75877,-0.396561,-0.756097,0.499048
-0.197309,0.435347,1.307597,-1.238725,0.497765,-0.185008,-1.313646,0.407795,-0.463788,-0.982147,-0.209881,0.007427,0.547946,-1.078012,0.07551,0.061511,-0.756764,-0.70996,-0.81592,0.14229
-0.139758,0.383882,1.429516,-0.947127,0.957552,-0.2337,-1.165889,-0.240241,-0.829311,-0.782506,-0.079749,0.098332,0.512785,-0.574034,-0.191348,0.260198,-0.589364,-0.292596,-0.538388,-0.037142
-0.017274,0.556072,1.29554,-0.804941,1.00275,-0.391457,-0.646703,-0.165443,-1.207828,-0.450585,0.031809,-0.085561,0.655302,-0.191815,-0.14504,0.335639,-0.504711,-0.176313,-0.380418,-0.053503
0.047931,0.768538,1.251776,-0.675649,0.773315,-0.077571,-0.691481,-0.266343,-0.928465,-0.353532,0.005045,0.160004,0.812558,-0.347111,-0.12406,-0.035316,-0.638011,-0.315887,-0.147963,-0.01308
0.00268,0.080124,0.028535,0.250282,0.027585,-0.076328,-0.135036,-0.144291,0.15689,-0.041616,-0.175671,-0.032258,0.440773,-0.05131,-0.270948,-0.024277,-0.287988,-0.299145,-0.072229,null


This func calculate suprise as a binary variable ("True" for positive and "False" for ngative)

In [105]:
def calc_suprise(tic_sym, df, period=10):
    periods = [50, 60, 70, 100, 1000]

    initial_delay = 1
    max_delay = 32

    eps = None

    delay = initial_delay
    for period in periods:
        
        try:
            company_ticker = yf.Ticker(tic_sym)
            eps = company_ticker.get_earnings_dates(limit=period)
            break
            
        except Exception as e:
            logging.error(f"Error calculating surprise for {tic_sym}: {e}")
        
        time.sleep(delay)
        delay = min(delay * 2, max_delay)
        
    if eps is None:
        return None, False  
    
    else:
        available_indices = list(eps.index)

        eps_dict = {}

        for col in df.columns:

            date_of_rep = pd.Timestamp(col, tz='America/New_York')
            closest_index = min(available_indices, key=lambda x: abs(x - date_of_rep))

            eps_surprise = eps.loc[closest_index, 'Surprise(%)']

            eps_dict[col] = eps_surprise

        eps_df = pl.DataFrame(eps_dict)   
        stacked_df = df.vstack(eps_df) 

        return stacked_df, True

In [124]:
calc_suprise(tic_sym, df)

ERROR:root:Error calculating surprise for ETR: 'Earnings Date'
ERROR:root:Error calculating surprise for ETR: 'Earnings Date'
ERROR:root:Error calculating surprise for ETR: 'Earnings Date'
ERROR:root:Error calculating surprise for ETR: 'Earnings Date'
ERROR:root:Error calculating surprise for ETR: 'Earnings Date'


(None, False)

In [126]:
def calc_suprise_sec_attempt(tic_sym, df):
    eps_dict = {}

    try:
        company_ticker = yf.Ticker(tic_sym)
        eps = company_ticker.earnings_dates

        available_indices = list(eps.index)

        eps_dict = {}

        retrived_dates = 0

        for col in sorted(df.columns, reverse=True):

            if available_indices:
                date_of_rep = pd.Timestamp(col, tz='America/New_York')

                closest_index = min(available_indices, key=lambda x: abs(x - date_of_rep))

                available_indices = [dt for dt in available_indices if dt < closest_index]

                eps_surprise = eps.loc[closest_index, 'Surprise(%)']

                eps_dict[col] = eps_surprise

                retrived_dates += 1
            else:
                eps_dict[col] = None

        failed_dates = len(df.columns) - retrived_dates

        eps_dict = {key: eps_dict[key] for key in sorted(eps_dict)}

    except Exception as e:
        logging.error(f"Error for second attempt surprise for {tic_sym}: {e}")

        for col in df.columns:
            eps_dict[col] = None

        failed_dates = len(df.columns)

    eps_df = pl.DataFrame(eps_dict)   
    stacked_df = df.vstack(eps_df)

    return stacked_df, failed_dates

calc_suprise_sec_attempt(tic_sym, df)

ERROR:root:Error for second attempt surprise for ETR: 'Earnings Date'


(shape: (6, 20)
 ┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
 │ 2019-02-2 ┆ 2019-05-0 ┆ 2019-08-0 ┆ 2019-11-0 ┆ … ┆ 2023-02-2 ┆ 2023-05-0 ┆ 2023-08-0 ┆ 2023-11- │
 │ 6         ┆ 3         ┆ 6         ┆ 5         ┆   ┆ 4         ┆ 4         ┆ 3         ┆ 02       │
 │ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
 │ f64       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
 ╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
 │ -0.255306 ┆ -0.16469  ┆ -0.1504   ┆ -0.226037 ┆ … ┆ -0.331853 ┆ -0.153611 ┆ -0.151989 ┆ -0.21845 │
 │           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 8        │
 │ -1263.0   ┆ -125.0    ┆ -188.0    ┆ -316.0    ┆ … ┆ -2242.0   ┆ -151.0    ┆ -214.0    ┆ -374.0   │
 │ 0.452186  ┆ 0.514083  ┆ 0.47309   ┆ 0.472521  ┆ … ┆ 0.455583  ┆

This is the general loop, that takes each key-value pair from df_dict_scores, applies computing_returns() to it, then verticaly stackes the result with orig df and stores it in the new dict

In [127]:
dict_for_anal = {}
row_names = pl.Series("row_names", ["lm_polarity", "lm_raw", "hiv4_polarity", "hiv4_raw", "doc_length", "eps_surprise", "2_day_reterns","3_day_reterns", "4_day_reterns", "5_day_reterns", "6_day_reterns", "7_day_reterns", "full_quarter_returns"])

company_fails = 0
doc_fails = 0
total_docs = 0

for tic_sym, df in df_dict_scores.items():
    total_docs += df.shape[1]
    print(tic_sym)
    
    regressors_df, sucssess = calc_suprise(tic_sym, df)

    if sucssess == False:
        company_fails +=1
        regressors_df, failed_dates = calc_suprise_sec_attempt(tic_sym, df)
        doc_fails += failed_dates

    returns_df = computing_returns(tic_sym, df)
    
    stacked_df = regressors_df.vstack(returns_df)
    stacked_df_with_indx = stacked_df.hstack([row_names])
    
    dict_for_anal[tic_sym] = stacked_df_with_indx

print(f'Company fails ratio: {company_fails/497}')
print(f'Documents fails ratio: {doc_fails/total_docs}')


A
AAL
AAPL
ABBV
ABNB
ABT
ACGL
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIZ
AJG
AKAM
ALB
ALGN
ALL
ALLE
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMZN
ANET
ANSS
AON
AOS
APA
APD
APH
APTV
ARE
ATO
AVB
AVGO
AVY
AWK
AXON
AXP
AZO
BA
BAC
BALL
BAX
BBWI
BBY
BDX
BEN
BF-B


ERROR:root:Error calculating surprise for BF-B: 'Earnings Date'
ERROR:root:Error calculating surprise for BF-B: 'Earnings Date'
ERROR:root:Error calculating surprise for BF-B: 'Earnings Date'
ERROR:root:Error calculating surprise for BF-B: 'Earnings Date'
ERROR:root:Error calculating surprise for BF-B: 'Earnings Date'
ERROR:root:Error for second attempt surprise for BF-B: 'Earnings Date'


BIIB


ERROR:root:Error calculating surprise for BIIB: 'Earnings Date'
ERROR:root:Error calculating surprise for BIIB: 'Earnings Date'
ERROR:root:Error calculating surprise for BIIB: 'Earnings Date'
ERROR:root:Error calculating surprise for BIIB: 'Earnings Date'
ERROR:root:Error calculating surprise for BIIB: 'Earnings Date'
ERROR:root:Error for second attempt surprise for BIIB: 'Earnings Date'


BIO


...

UNH


ERROR:root:Error calculating surprise for UNH: 'Earnings Date'
ERROR:root:Error calculating surprise for UNH: 'Earnings Date'


UNP
UPS
URI
USB
V
VICI
VLO
VLTO
VMC
VRSK
VRSN
VRTX
VST
VTR
VTRS
VZ
WAB
WAT
WBA
WBD
WDC
WEC
WELL
WFC
WM
WMB
WMT
WRB
WRK
WST
WTW
WY
WYNN
XEL
XOM
XYL
YUM
ZBH
ZBRA
ZTS


ERROR:root:Error calculating surprise for ZTS: 'Earnings Date'
ERROR:root:Error calculating surprise for ZTS: 'Earnings Date'
ERROR:root:Error calculating surprise for ZTS: 'Earnings Date'
ERROR:root:Error calculating surprise for ZTS: 'Earnings Date'
ERROR:root:Error calculating surprise for ZTS: 'Earnings Date'
ERROR:root:Error for second attempt surprise for ZTS: 'Earnings Date'


Company fails ratio: 0.0
Documents fails ratio: 0.0


In [128]:
dict_for_anal["AEE"]

2019-02-26,2019-05-10,2019-08-02,2019-11-08,2020-02-28,2020-05-11,2020-08-07,2020-11-05,2021-02-22,2021-05-10,2021-08-06,2021-11-04,2022-02-23,2022-05-06,2022-08-08,2022-11-04,2023-02-22,2023-05-05,2023-08-03,2023-11-09,row_names
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
-0.211316,-0.202797,-0.192547,-0.19604,-0.242243,-0.234177,-0.320207,-0.306324,-0.266361,-0.23224,-0.246114,-0.213582,-0.238938,-0.242574,-0.227633,-0.214206,-0.224048,-0.192982,-0.2,-0.167592,"""lm_polarity"""
-366.0,-87.0,-93.0,-99.0,-406.0,-148.0,-309.0,-310.0,-639.0,-170.0,-190.0,-195.0,-594.0,-196.0,-201.0,-193.0,-559.0,-154.0,-167.0,-151.0,"""lm_raw"""
0.470266,0.508287,0.502341,0.520712,0.479986,0.50509,0.433089,0.439725,0.460614,0.464941,0.453013,0.46332,0.476764,0.477736,0.452534,0.44806,0.468905,0.456778,0.437378,0.442765,"""hiv4_polarity"""
4286.0,1288.0,1395.0,1521.0,4065.0,1439.0,1890.0,2050.0,5368.0,1724.0,1827.0,2160.0,5899.0,1899.0,2045.0,2148.0,6062.0,1934.0,2015.0,2139.0,"""hiv4_raw"""
58108.0,17100.0,19018.0,20193.0,54567.0,19648.0,31067.0,33618.0,75857.0,26740.0,29809.0,34186.0,81542.0,28414.0,33272.0,35349.0,85344.0,29831.0,33058.0,34479.0,"""doc_length"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
-0.084389,0.761784,-0.11339,0.415438,0.362239,-0.02873,0.261051,-0.942135,0.005088,0.14359,0.521448,-0.211856,0.07061,1.608919,0.240996,0.149262,-0.634927,-0.271307,-0.990158,-1.021736,"""4_day_reterns"""
0.061369,0.695965,0.023151,0.469176,0.675814,-0.003196,0.400904,-0.898,-0.173651,-0.054816,0.286068,0.036607,-0.003176,0.899509,0.042316,-0.028312,-0.747312,-0.359165,-0.687648,-0.514444,"""5_day_reterns"""
0.064116,0.91196,0.271545,0.381931,0.702493,0.115399,0.224892,-0.848677,-0.220381,-0.145381,0.41466,0.079057,-0.052914,0.60167,0.122525,-0.498796,-0.70567,-0.52526,-0.570334,-0.699415,"""6_day_reterns"""


This func takes as input key-value pair (company name and dataframe with sentiment scores and returns), regress each retern (2_days, 3_days, etc) on polarity score and raw sentiment score. Then it collects the slopes (beta-coefficient) of returns regressed on each score in the separate polars series (in the loop they will be staked in dataframe)

In [129]:
def add_quarter_dummy(company_df_pd, var_names):
    
    dates = pd.to_datetime(company_df_pd.columns)
   
    date_series = pd.Series(dates.quarter, index=dates)
   
    dummies = pd.get_dummies(date_series, prefix="Q")
    dummies.index = dummies.index.strftime('%Y-%m-%d')
    
    all_quarters = ["Q_1", "Q_2", "Q_3", "Q_4"]
    dummies = dummies.reindex(columns=all_quarters, fill_value=False)
   
    dummies = dummies.drop(columns=["Q_4"])
    
    dummies = dummies.transpose()
    df_wit_dum = pd.concat([company_df_pd, dummies])
    
    df_wit_dum = df_wit_dum.transpose()
   
    for col in df_wit_dum.columns:
        df_wit_dum[col] = pd.to_numeric(df_wit_dum[col])

    boolean_cols = ['Q_1', 'Q_2', 'Q_3']
    df_wit_dum[boolean_cols] = df_wit_dum[boolean_cols].astype(int)

    res_col_names = var_names + boolean_cols

    df_wit_dum.columns = res_col_names

    return df_wit_dum

    

In [131]:
def fit_regression(y, x):

    x = sm.add_constant(x)
    model = sm.OLS(y, x, missing='drop').fit()
    
    slope = model.params.iloc[1]
    
    return slope

In [218]:
def regression_slopes(df_wit_dum, X_names, company_name):
    
    for col in df_wit_dum.select_dtypes(include=['bool']).columns:
        df_wit_dum[col] = df_wit_dum[col].astype(int)
        
    df_wit_dum = df_wit_dum.apply(pd.to_numeric, errors='coerce')

    # x in range(0, 4) represents set of independent variables; 4th column represents control variable for document length; -3, -2, -1 columns represent dummy variables for quarters
    X = [df_wit_dum.iloc[:, [x, 4, 5, -3, -2, -1]] for x in range(0, 4)]

    Y = [df_wit_dum.iloc[:, [y]] for y in range(6, 13)]

    slopes_dict = {}

    for i, x in enumerate(X):

        name_of_score = X_names[i][0].split()[0]
        slopes_name = f'{name_of_score}_slopes'
        #print( slopes_name)
        
        slopes = []
        for y in Y:
    
            slope = fit_regression(y, x)
            slopes.append(slope)

        slopes_dict[slopes_name] = pl.Series(company_name, slopes)
    
    return slopes_dict

This is the func to check if there is enough data in single company df

In [ ]:
def check_17_nonnull(df):
    df = df.transpose()

    counts = df.count()
    mask = counts < 17
    if mask.sum_horizontal().item() > 0:
        return False
    else:
        return True

This is the general loop that takes dict_for_anal whith scores and returns and for each company in the dict it calculates dict with four dfs: first one contain regression slopes for lm_polarity scores, second one for lm_raw scores and so on for each regressor. Serieses stacked in the separete dataframes (for raw and for polarity scores)

In [219]:
def compute_all_slopes(dict_for_anal):

    slopes_df_dict = {
        "lm_polarity_slopes_df" : pl.DataFrame(),
        "lm_raw_slopes_df" : pl.DataFrame(),
        "hiv4_polarity_slopes_df" : pl.DataFrame(),
        "hiv4_raw_slopes_df" : pl.DataFrame()
    }

    fails = 0

    for company_name, company_df in dict_for_anal.items(): 

        X_names = [company_df.select(pl.col("row_names")).row(x) for x in range(0, 4)]

        var_names = company_df.get_column("row_names").to_list()
        company_df_woi = company_df.select(pl.exclude("row_names"))

        company_df_woi = company_df_woi.to_pandas()
        print(company_name)

        df_wit_dum = add_quarter_dummy(company_df_woi, var_names)

        enough_data = check_17_nonnull(company_df)

        if enough_data == True:

            slopes_dict = regression_slopes(df_wit_dum, X_names, company_name)
    
            for x in X_names:
                
                name_of_score = x[0].split()[0]
                slopes_name = f'{name_of_score}_slopes'
                
                df_name = f"{name_of_score}_slopes_df"
                
                slopes_df_dict[df_name] = slopes_df_dict[df_name].hstack([slopes_dict[slopes_name]])
        else:
            fails +=1
    print(f'fails rate: {fails/len(dict_for_anal)}')

    return slopes_df_dict
        

In [221]:
slopes_df_dict = compute_all_slopes(dict_for_anal)

A
AAL
AAPL
ABBV
ABNB
ABT
ACGL
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIZ
AJG
AKAM
ALB
ALGN
ALL
ALLE
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMZN
ANET
ANSS
AON
AOS
APA
APD
APH
APTV
ARE
ATO
AVB
AVGO
AVY
AWK
AXON
AXP
AZO
BA
BAC
BALL
BAX
BBWI
BBY
BDX
BEN
BF-B
BIIB
BIO
BK
BKNG
BKR
BLDR
BLK
BMY
BR
BRK-B
BRO
BSX
BWA
BX
BXP
C
CAG
CAH
CARR
CAT
CB
CBOE
CBRE
CCI
CCL
CDNS
CDW
CE
CEG
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNC
CNP
COF
COO
COP
COR
COST
CPAY
CPB
CPRT
CPT
CRL
CRM
CSCO
CSGP
CSX
CTAS
CTLT
CTRA
CTSH
CTVA
CVS
CVX
CZR
D
DAL
DAY
DD
DE
DECK
DFS
DG
DGX
DHI
DHR
DIS
DLR
DLTR
DOC
DOV
DOW
DPZ
DRI
DTE
DUK
DVA
DVN
DXCM
EA
EBAY
ECL
ED
EFX
EG
EIX
EL
ELV
EMN
EMR
ENPH
EOG
EPAM
EQIX
EQR
EQT
ES
ESS
ETN
ETR
ETSY
EVRG
EW
EXC
EXPD
EXPE
EXR
F
FANG
FAST
FCX
FDS
FDX
FE
FFIV
FI
FICO
FIS
FITB
FMC
FOXA
FRT
FSLR
FTNT
FTV
GD
GE
GEHC
GEN
GILD
GIS
GL
GLW
GM
GNRC
GOOGL
GPC
GPN
GRMN
GS
GWW
HAL
HAS
HBAN
HCA
HD
HES
HIG
HII
HLT
HOLX
HON
HPE
HPQ
HRL
HSIC
HST
HSY
HUBB
HUM
HWM
IBM
ICE
IDXX
IEX
IFF
ILMN
IN

As an output from previous functions we have dataframes with regression slopes of different returns on raw, polarity scores and Loughran-McDonald and Harvard dictionaries. Each col of these dfs contain regression slopes for particular company. First row contain slope for 2_days return, second row contain slope for 3_days return, etc.  I have decided to check what is the proportion of positive regression slopes for each return timeframe (2_days, 3_days, etc). For that purpose i will transpose each of these dfs, apply condition > 0, then verticaly sum boolean values and obtain the prorortion of positive slopes for each time frame. 

In [222]:
def calc_efficiency_metrics(slopes_df):

    df_tranposed = slopes_df.transpose()

    res = df_tranposed.select(pl.all() > 0).sum()
    total_num_of_slopes = slopes_df.shape[1]

    positive_slopes = [res.select(pl.col(res.columns[x]).gather(0)).item() for x in range(slopes_df.shape[0])]

    for counter, x in enumerate(positive_slopes):
        print(f'Prorortion of positive {counter + 2}_day returns: {x/total_num_of_slopes}')

This loop apply calc_efficiency_metrics function to each item of slopes_df_dict where each item is a dataframe which regression slopes for each regressor.

In [223]:
for key, slopes_df in slopes_df_dict.items():
    print("\n")
    print(f'Metrics for: {key}')
    calc_efficiency_metrics(slopes_df)



Metrics for: lm_polarity_slopes_df
Prorortion of positive 2_day returns: 0.4817987152034261
Prorortion of positive 3_day returns: 0.4860813704496788
Prorortion of positive 4_day returns: 0.4925053533190578
Prorortion of positive 5_day returns: 0.4925053533190578
Prorortion of positive 6_day returns: 0.4925053533190578
Prorortion of positive 7_day returns: 0.48394004282655245
Prorortion of positive 8_day returns: 0.4603854389721627


Metrics for: lm_raw_slopes_df
Prorortion of positive 2_day returns: 0.4732334047109208
Prorortion of positive 3_day returns: 0.4668094218415418
Prorortion of positive 4_day returns: 0.4689507494646681
Prorortion of positive 5_day returns: 0.4518201284796574
Prorortion of positive 6_day returns: 0.47109207708779444
Prorortion of positive 7_day returns: 0.4796573875802998
Prorortion of positive 8_day returns: 0.44967880085653106


Metrics for: hiv4_polarity_slopes_df
Prorortion of positive 2_day returns: 0.4689507494646681
Prorortion of positive 3_day retur

Positive slopes means that sentiment scores and returns mooves in the same direction (hense, they are working as predictors of stock price movement). As the result of this preliminary analisis we can say that sentiment scores of quarterly reports are not very good predictors of stock movments. Positive proportion of this slopes are close to 0.5, which means that on average in our sample there are 50/50 chance that scores and returns are mooving in the same direction.

As we can see that, despite visual conclusion that weighted scores are more "smooth", they barely outperform raw scores, but at least they are not worse, meaning that at each time frame (except 5_days) the proportion of positive regression slopes of weighted scores => proportion of positive regression slopes of raw scores.

This is the separate fit() function

In [224]:
def fit_rob_se(y, x):

    x = sm.add_constant(x)
    model = sm.OLS(y, x, missing='drop').fit()
    robust_se = model.get_robustcov_results(cov_type='HC0')
    
    return robust_se.summary()

This func is to get parametrs of the model to df

In [225]:
def extract_params(summary):
    
    coef_table = summary.tables[1].as_html()
    coef_table_io = StringIO(coef_table)
    df = pd.read_html(coef_table_io, header=0, index_col=0)[0]

    df.columns = ['coef', 'std err', 't', 'P>|t|', '[0.025', '0.975]']
    
    df = df.reset_index()
    df = df.rename(columns={'index': 'Variable'})

    filtered_df = df[df['P>|t|'] < 0.05]
    
    return filtered_df


This is a separate regression function

In [226]:
def regression_single_slope(df_wit_dum, X_names, Y_names):

    for col in df_wit_dum.select_dtypes(include=['bool']).columns:
        df_wit_dum[col] = df_wit_dum[col].astype(int)

    df_wit_dum = df_wit_dum.apply(pd.to_numeric, errors='coerce')

    # x in range(0, 4) represents set of independent variables; 4th column represents control variable for document length; -3, -2, -1 columns represent dummy variables for quarters
    X = [df_wit_dum.iloc[:, [x, 4, 5, -14, -13, -12, -11, -10, -9,-8 ,-7, -6, -5 ,-4, -3, -2, -1]] for x in range(0, 4)]

    Y = [df_wit_dum.iloc[:, [y]] for y in range(6, 13)]

    for i, x in enumerate(X):

        name_of_score = X_names[i][0]

        for j, y in enumerate(Y):
            returns_timeframe = Y_names[j][0]
            
            summary = fit_rob_se(y, x)
            params_df = extract_params(summary)
            if not params_df.empty:
                print(name_of_score, returns_timeframe)
                print(params_df, '\n') 

This func add sector dummy

In [141]:
def addsector_value(company_name, df_wit_dum):
    ticker = yf.Ticker(company_name)
    company_info = ticker.info
    sector = company_info.get('sector', 'Sector information not available')
    df_wit_dum['sector'] = sector
    return df_wit_dum
    

This func stackes all data in one df, and calculate the single slope for each time frame

In [151]:
def compute_signl_slope(dict_for_anal):

    df_list = []
    
    for company_name in dict_for_anal:


        X_names = [dict_for_anal[company_name].select(pl.col("row_names")).row(x) for x in range(0, 4)]
        Y_names = [dict_for_anal[company_name].select(pl.col("row_names")).row(x) for x in range(6, 13)] 

        var_names = dict_for_anal[company_name].get_column("row_names").to_list()

        company_df_woi = dict_for_anal[company_name].select(pl.exclude("row_names"))
        company_df_pd = company_df_woi.to_pandas(use_pyarrow_extension_array=True)

        df_wit_dum = add_quarter_dummy(company_df_pd, var_names)
        df_wit_sector = addsector_value(company_name, df_wit_dum)
        #print(df_wit_sector)

        df_list.append(df_wit_sector)
        
    total_df = pd.concat(df_list, ignore_index=True)
    total_df_with_dummies = pd.get_dummies(total_df, columns=['sector'])
    print(total_df_with_dummies)
    
    return total_df_with_dummies, X_names, Y_names
    #regression_single_slope(total_df, X_names, Y_names)


In [152]:
total_df_with_dummies, X_names, Y_names = compute_signl_slope(dict_for_anal)


      lm_polarity  lm_raw  hiv4_polarity  hiv4_raw  doc_length  eps_surprise  \
0       -0.329588  -352.0       0.327818    1387.0     25966.0        0.0402   
1       -0.321787  -353.0       0.318681    1421.0     27591.0       -0.0075   
2       -0.325390  -355.0       0.338225    1521.0     27714.0        0.0517   
3       -0.266055  -493.0       0.400181    3540.0     53169.0        0.0414   
4       -0.322245  -310.0       0.369223    1454.0     24310.0        0.0010   
...           ...     ...            ...       ...         ...           ...   
9825    -0.396226  -273.0       0.406832    1310.0     19459.0           NaN   
9826    -0.291170  -709.0       0.439736    4535.0     59370.0           NaN   
9827    -0.296748  -146.0       0.463665    1174.0     15299.0           NaN   
9828    -0.285171  -150.0       0.453390    1284.0     17288.0           NaN   
9829    -0.267760  -147.0       0.448852    1290.0     17537.0           NaN   

      2_day_reterns  3_day_reterns  4_d

In [227]:
regression_single_slope(total_df_with_dummies, X_names, Y_names)

lm_polarity 2_day_reterns
            Variable      coef   std err      t  P>|t|        [0.025    0.975]
2         doc_length  0.000003  0.000001  2.566  0.010  7.930000e-07  0.000006
4                Q_1 -0.157100  0.061000 -2.589  0.010 -2.760000e-01 -0.038000
5                Q_2 -0.191500  0.056000 -3.443  0.001 -3.010000e-01 -0.082000
6                Q_3 -0.134600  0.054000 -2.471  0.013 -2.410000e-01 -0.028000
17  sector_Utilities -0.213000  0.064000 -3.336  0.001 -3.380000e-01 -0.088000 

lm_polarity 3_day_reterns
            Variable      coef   std err      t  P>|t|        [0.025    0.975]
2         doc_length  0.000002  0.000001  2.040  0.041  8.110000e-08  0.000004
4                Q_1 -0.098200  0.046000 -2.125  0.034 -1.890000e-01 -0.008000
5                Q_2 -0.147700  0.042000 -3.483  0.000 -2.310000e-01 -0.065000
6                Q_3 -0.092400  0.041000 -2.271  0.023 -1.720000e-01 -0.013000
17  sector_Utilities -0.138400  0.048000 -2.869  0.004 -2.330000e-01 -0.04400

jupyter nbconvert --to html Parsim-sec\src\Analysis\excess_ret_500.ipynb